Cells analysis


In [8]:
import os
import numpy as np
import tifffile as tiff
from skimage.morphology import label



In [15]:

def save_cells_with_volumes(segmented_stack_path, output_dir, voxel_size):
    """
    Save each segmented cell and calculate its volume based on voxel dimensions.
    
    :param segmented_stack_path: str, path to the segmented 3D stack (can be .npy or .tif).
    :param output_dir: str, directory to save the individual cell files.
    :param voxel_size: tuple, (x, y, z) dimensions of a voxel in microns.
    """
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Load segmented stack
    if segmented_stack_path.endswith('.npy'):
        segmented_stack = np.load(segmented_stack_path)
    elif segmented_stack_path.endswith('.tif'):
        segmented_stack = tiff.imread(segmented_stack_path)
    else:
        raise ValueError("Unsupported file format. Use .npy or .tif.")

    # Get unique cell IDs (excluding background ID 0)
    unique_ids = np.unique(segmented_stack)
    unique_ids = unique_ids[unique_ids > 0]

    print(f"Found {len(unique_ids)} cells in the segmented stack.")

    # Calculate voxel volume in microns^3
    voxel_volume = voxel_size[0] * voxel_size[1] * voxel_size[2]

    # Process each cell
    for cell_id in unique_ids:
        print(f"Processing cell ID: {cell_id}")
        
        # Create a mask for the current cell
        cell_mask = segmented_stack == cell_id
        
        # Isolate the cell
        cell_volume = cell_mask * cell_id  # Keep the cell ID for context
        
        # Calculate the physical volume of the cell
        physical_volume = np.sum(cell_mask) * voxel_volume
        print(f"Cell ID {cell_id} volume: {physical_volume:.2f} microns^3")
        
        # Save as .npy
        npy_path = os.path.join(output_dir, f'cell_{cell_id}.npy')
        np.save(npy_path, cell_volume)
        
        # Save as .tif
        tif_path = os.path.join(output_dir, f'cell_{cell_id}.tif')
        tiff.imwrite(tif_path, cell_volume.astype(np.uint16))
        
        print(f"Saved Cell ID {cell_id}: {npy_path} and {tif_path}")



In [16]:
#Usage
if __name__ == "__main__":
    input_image_path = "/Users/nayeb/Desktop/UCSF_LAB_PRACTICE/2d_pipeline/TC_17_Cortical2-1-2_augmented_11_segmented.tif"
    output_directory = "/Users/nayeb/Desktop/UCSF_LAB_PRACTICE/single_cell"
    voxel_size = (0.5, 0.5, 1.0)  # in microns (x, y, z)
    npy_path = "/Users/nayeb/Desktop/UCSF_LAB_PRACTICE/2d_pipeline"
    segmented_stack_path = f"{npy_path}/TC_17_Cortical2-1-2_augmented_11_segmented.npy"
    save_cells_with_volumes(segmented_stack_path, "/Users/nayeb/Desktop/UCSF_LAB_PRACTICE/single_cell", voxel_size)

Found 17 cells in the segmented stack.
Processing cell ID: 1
Cell ID 1 volume: 123.00 microns^3
Saved Cell ID 1: /Users/nayeb/Desktop/UCSF_LAB_PRACTICE/single_cell/cell_1.npy and /Users/nayeb/Desktop/UCSF_LAB_PRACTICE/single_cell/cell_1.tif
Processing cell ID: 2
Cell ID 2 volume: 2732.75 microns^3
Saved Cell ID 2: /Users/nayeb/Desktop/UCSF_LAB_PRACTICE/single_cell/cell_2.npy and /Users/nayeb/Desktop/UCSF_LAB_PRACTICE/single_cell/cell_2.tif
Processing cell ID: 3
Cell ID 3 volume: 96.00 microns^3
Saved Cell ID 3: /Users/nayeb/Desktop/UCSF_LAB_PRACTICE/single_cell/cell_3.npy and /Users/nayeb/Desktop/UCSF_LAB_PRACTICE/single_cell/cell_3.tif
Processing cell ID: 4
Cell ID 4 volume: 3758.25 microns^3
Saved Cell ID 4: /Users/nayeb/Desktop/UCSF_LAB_PRACTICE/single_cell/cell_4.npy and /Users/nayeb/Desktop/UCSF_LAB_PRACTICE/single_cell/cell_4.tif
Processing cell ID: 5
Cell ID 5 volume: 2472.00 microns^3
Saved Cell ID 5: /Users/nayeb/Desktop/UCSF_LAB_PRACTICE/single_cell/cell_5.npy and /Users/naye

Recombine all to compare 

In [14]:
import os
import numpy as np
import tifffile as tiff

# Path to the directory containing single-cell TIFF files
single_cell_dir = "/Users/nayeb/Desktop/UCSF_LAB_PRACTICE/single_cell"

# Path to the original image (to get dimensions)
original_image_path = "/Users/nayeb/Desktop/UCSF_LAB_PRACTICE/2d_pipeline/TC_17_Cortical2-1-2_augmented_11.tiff"

# Output path for the combined z-stack TIFF
combined_tiff_path = "/Users/nayeb/Desktop/UCSF_LAB_PRACTICE/combined_zstack.tif"

# Load the original image to get the shape
original_image = tiff.imread(original_image_path)
z_dim, height, width = original_image.shape

# Initialize an empty array for the combined z-stack
combined_stack = np.zeros((z_dim, height, width), dtype=np.uint16)

# Process each single-cell TIFF
cell_files = sorted([os.path.join(single_cell_dir, f) for f in os.listdir(single_cell_dir) if f.endswith('.tif')])

for cell_file in cell_files:
    print(f"Adding {cell_file} to z-stack")
    cell_data = tiff.imread(cell_file)
    
    # Ensure the dimensions match the original image
    if cell_data.shape != (z_dim, height, width):
        raise ValueError(f"Dimension mismatch: {cell_file} has shape {cell_data.shape}, expected {(z_dim, height, width)}")
    
    # Add the cell to the combined stack (maintaining unique IDs)
    combined_stack += cell_data.astype(np.uint16)

# Save the combined z-stack as a multi-layer TIFF
tiff.imwrite(combined_tiff_path, combined_stack)
print(f"Combined z-stack saved to: {combined_tiff_path}")


Adding /Users/nayeb/Desktop/UCSF_LAB_PRACTICE/single_cell/cell_10.tif to z-stack
Adding /Users/nayeb/Desktop/UCSF_LAB_PRACTICE/single_cell/cell_100.tif to z-stack
Adding /Users/nayeb/Desktop/UCSF_LAB_PRACTICE/single_cell/cell_101.tif to z-stack
Adding /Users/nayeb/Desktop/UCSF_LAB_PRACTICE/single_cell/cell_102.tif to z-stack
Adding /Users/nayeb/Desktop/UCSF_LAB_PRACTICE/single_cell/cell_103.tif to z-stack
Adding /Users/nayeb/Desktop/UCSF_LAB_PRACTICE/single_cell/cell_104.tif to z-stack
Adding /Users/nayeb/Desktop/UCSF_LAB_PRACTICE/single_cell/cell_105.tif to z-stack
Adding /Users/nayeb/Desktop/UCSF_LAB_PRACTICE/single_cell/cell_106.tif to z-stack
Adding /Users/nayeb/Desktop/UCSF_LAB_PRACTICE/single_cell/cell_107.tif to z-stack
Adding /Users/nayeb/Desktop/UCSF_LAB_PRACTICE/single_cell/cell_108.tif to z-stack
Adding /Users/nayeb/Desktop/UCSF_LAB_PRACTICE/single_cell/cell_109.tif to z-stack
Adding /Users/nayeb/Desktop/UCSF_LAB_PRACTICE/single_cell/cell_11.tif to z-stack
Adding /Users/naye